# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up Turi Create

In [3]:
# import turicreate
import pandas as pd
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [ ]:
sales = turicreate.SFrame('home_data.sframe/')

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let Turi Create pick a random seed for you).  

In [6]:
#train_data,test_data = sales.random_split(.8,seed=0)

In [4]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
train_data = pd.read_csv('kc_house_train_data.csv',dtype=dtype_dict)
test_data = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [ ]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = turicreate.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                    validation_set = None)

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [ ]:
example_weight_summary = example_model.coefficients
print example_weight_summary

# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing Turi Create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [ ]:
example_predictions = example_model.predict(train_data)
print example_predictions[0] # should be 271789.505878

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [26]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    y_hat = model.predict(data)
    # Then compute the residuals/errors
    # Then square and add them up
    RSS = sum((outcome-y_hat)**2)
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [ ]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print rss_example_train # should be 2.7376153833e+14

# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [16]:
from math import log
import statsmodels.api as sm


Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [8]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [12]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))
train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data ['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [14]:
mean_bedrooms_squared = test_data ['bedrooms_squared'].mean()
mean_bed_bath_rooms = test_data ['bed_bath_rooms'].mean()
mean_log_sqft_living = test_data ['log_sqft_living'].mean()
mean_lat_plus_long = test_data ['lat_plus_long'].mean()
print(mean_bedrooms_squared,mean_bed_bath_rooms,mean_log_sqft_living,mean_lat_plus_long)

12.4466777015843 7.5039016315913925 7.550274679645921 -74.65333355403185


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [15]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [17]:
# Learn the three models: (don't forget to set validation_set = None)
y = train_data['price']

X1 = train_data[model_1_features]
X1 = sm.add_constant(X1)
ols_1 = sm.OLS(y, X1).fit()

X2 = train_data[model_2_features]
X2 = sm.add_constant(X2)
ols_2 = sm.OLS(y, X2).fit()


X3 = train_data[model_3_features]
X3 = sm.add_constant(X3)
ols_3 = sm.OLS(y, X3).fit()




In [19]:
# Examine/extract each model's coefficients:
ols_1.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.593
Model:                            OLS   Adj. R-squared:                  0.592
Method:                 Least Squares   F-statistic:                     5056.
Date:                Wed, 19 Jan 2022   Prob (F-statistic):               0.00
Time:                        22:16:11   Log-Likelihood:            -2.3973e+05
No. Observations:               17384   AIC:                         4.795e+05
Df Residuals:                   17378   BIC:                         4.795e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -6.908e+07   1.65e+06    -41.940      0.000   -7.23e+07   -6.58e+07
sqft_living   312.2586      3.183     98.097      0.000     306.019     318.498
bedrooms    -5.959e+04   2482.861    -23.999      0.000   -6.45e+04   -5.47e+04
bathrooms    1.571e+04   3587.158      4.379      0.000    8675.552    2.27e+04
lat          6.586e+05   1.31e+04     50.286      0.000    6.33e+05    6.84e+05
long        -3.094e+05   1.33e+04    -23.331      0.000   -3.35e+05   -2.83e+05
==============================================================================
Omnibus:                    12962.917   Durbin-Watson:                   2.008
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           615844.364
Skew:                           3.114   Prob(JB):                         0.00
Kurtosis:                      31.486   Cond. No.                     2.10e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.1e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [20]:
ols_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.597
Model:                            OLS   Adj. R-squared:                  0.596
Method:                 Least Squares   F-statistic:                     4283.
Date:                Wed, 19 Jan 2022   Prob (F-statistic):               0.00
Time:                        22:16:19   Log-Likelihood:            -2.3965e+05
No. Observations:               17384   AIC:                         4.793e+05
Df Residuals:                   17377   BIC:                         4.794e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -6.687e+07   1.65e+06    -40.584      0.000   -7.01e+07   -6.36e+07
sqft_living      306.6101      3.197     95.909      0.000     300.344     312.876
bedrooms       -1.134e+05   4797.612    -23.646      0.000   -1.23e+05   -1.04e+05
bathrooms      -7.146e+04   7552.563     -9.462      0.000   -8.63e+04   -5.67e+04
lat             6.548e+05    1.3e+04     50.230      0.000    6.29e+05     6.8e+05
long           -2.943e+05   1.32e+04    -22.218      0.000    -3.2e+05   -2.68e+05
bed_bath_rooms  2.558e+04   1953.134     13.097      0.000    2.18e+04    2.94e+04
==============================================================================
Omnibus:                    12343.124   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           533516.100
Skew:                           2.916   Prob(JB):                         0.00
Kurtosis:                      29.506   Cond. No.                     2.11e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.11e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [21]:
ols_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.620
Model:                            OLS   Adj. R-squared:                  0.620
Method:                 Least Squares   F-statistic:                     3539.
Date:                Wed, 19 Jan 2022   Prob (F-statistic):               0.00
Time:                        22:16:28   Log-Likelihood:            -2.3913e+05
No. Observations:               17384   AIC:                         4.783e+05
Df Residuals:                   17375   BIC:                         4.784e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -6.204e+07   1.61e+06    -38.449      0.000   -6.52e+07   -5.89e+07
sqft_living        529.4228      7.691     68.836      0.000     514.348     544.498
bedrooms          3.451e+04   9385.891      3.677      0.000    1.61e+04    5.29e+04
bathrooms         6.706e+04   1.08e+04      6.219      0.000    4.59e+04    8.82e+04
lat               5.341e+05   8907.125     59.962      0.000    5.17e+05    5.52e+05
long             -4.068e+05   9007.995    -45.154      0.000   -4.24e+05   -3.89e+05
bed_bath_rooms   -8570.5044   2855.965     -3.001      0.003   -1.42e+04   -2972.526
bedrooms_squared -6788.5867   1493.406     -4.546      0.000   -9715.813   -3861.360
log_sqft_living  -5.618e+05   1.75e+04    -32.014      0.000   -5.96e+05   -5.27e+05
lat_plus_long     1.273e+05   6450.344     19.741      0.000    1.15e+05     1.4e+05
==============================================================================
Omnibus:                    10094.582   Durbin-Watson:                   2.005
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           319322.176
Skew:                           2.240   Prob(JB):                         0.00
Kurtosis:                      23.513   Cond. No.                     1.87e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.59e-24. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

***positive***



**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

***negative***


Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [41]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
RSS_1 = get_residual_sum_of_squares(ols_1,X1,y)
RSS_2 = get_residual_sum_of_squares(ols_2,X2,y)
RSS_3 = get_residual_sum_of_squares(ols_3,X3,y)
print('RSS of model 1 , model 2 and model 3 are: ', "{:e}".format(RSS_1),"{:e}".format(RSS_2),"{:e}".format(RSS_3))

RSS of model 1 , model 2 and model 3 are:  9.678800e+14 9.584196e+14 9.034365e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

***model 3***

Now compute the RSS on on TEST data for each of the three models.

In [43]:
# Compute the RSS on TESTING data for each of the three models and record the values:
y_test = test_data ['price']
X1_test = test_data [model_1_features]
X1_test = sm.add_constant(X1_test)
X2_test = test_data [model_2_features]
X2_test = sm.add_constant(X2_test)
X3_test = test_data [model_3_features]
X3_test = sm.add_constant(X3_test)
RSS_1_test = get_residual_sum_of_squares(ols_1,X1_test,y_test)
RSS_2_test = get_residual_sum_of_squares(ols_2,X2_test,y_test)
RSS_3_test = get_residual_sum_of_squares(ols_3,X3_test,y_test)
print('RSS of model 1 , model 2 and model 3 are: ', "{:e}".format(RSS_1_test),"{:e}".format(RSS_2_test),"{:e}".format(RSS_3_test))

RSS of model 1 , model 2 and model 3 are:  2.255005e+14 2.233775e+14 2.592363e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.

***model1***